# From sky to images

## What does ALMA observes?

Let us say there is an interesting object in the sky, and we wish to study its spatial brightness distribution. For simplicity, let us assume we are observing a portion of the sky small enough such that the sky can be described as a flat surface (this is the case for most ALMA observations of planet-forming disks). Our object of study will have an intensity distribution over the 2D surface defined as $I(l,m)$, where $(l,m)$ are the sky spatial coordinates. From an observational point of view, **our goal is to recover the function** $I$ **as accurately as possible**.

As [ALMA](https://www.almaobservatory.org/en/home/) is an [interferometer](https://en.wikipedia.org/wiki/Astronomical_interferometer), it does not directly observe the sky brightness distribution (as most optical and near infrared cameras). Instead, **ALMA observes the visibility function** $V$ **of the sky intensity**, which is the Fourier Transform of the intensity distribution:

$$
  {V}(u, v) = \int \int {I}(l,m) \, e^{-2\pi i (ul + vm)}\,\text{d}l\,\text{d}m
$$

If we knew $V$ for every possible $(u,v)$, then recovering $I$ would be as simple as calculating the inverse Fourier Transform of $V$. With interferometers, that is not the case. For a given set of antennas, the **baselines** of an observation are all the possible combinations of antenna pairs. Each baseline $i$ will measure the value of $V_i$ in a given coordinate $(u_i,v_i)$, therefore, an interferometer with $N$ baselines will sample the values $\{V(u_i, v_i)\}_{i=0}^N$ only for a discrete set of positions $\left\{(u_i,v_i)\right\}_{i=0}^N$. This set of coordinates in the uv-plane will be called **uv-coverage** (see panel (c) of **Figure 1** for an example of an incompletely sampled visibility space).

## Reconstructing the sky brightness distribution

As we only have an incomplete measurement of the visibility function $V$, we cannot directly recover the sky intensity distribution $I$ with an inverse Fourier Transform. Instead, we will need to assume values of $V$ in the spatial frequencies $(u,v)$ where we do not have measurements.

Although there are several different algorithms to reconstruct $I$ from an incompletely sampled $V$, all of them rely on creating a model $I_{\text{mod}}$. The visibility function of this model matches $\{V(u_i, v_i)\}_{i=0}^N$ by construction or by the minimization of a given variable (such as $\chi^2$). The whole process from observing to model reconstruction is shown in the following **Figure 1**:

<img src="./sky2obs.png">
<center> Figure 1: An object in the sky has a certain visibility representation, given by its Fourier Transform. ALMA only samples the visibility representation in a discrete set of spatial frecuencies, and we need to reconstruct a model to recover the sky brightness distribution. </center>

## Why do we convolve the models with Gaussians?

The longest baselines of an interferometer set the maximum angular resolution of a particular observation. An interferometer does not sample details in spatial frequencies higher than the longest baselines can access, thus a given observation will have no information about them.

On the other hand, the reconstructed models have information stored in pixels (the minimum spatial unit of an image). These pixels, however, are much smaller than the physical angular resolution limit an observation can achieve, meaning that the information of intensity variation from one pixel to another is contained in a region of the visibility function that was not sampled by the observation. In other words, the observation does not strongly constrain what individual pixels do in a model.

One way to deal with this super-resolution behavior of the pixels in a model is to convolve it with a Gaussian representative of the observation PSF (point spread function). In this way, the information is deleted in spatial scales smaller than the angular resolution, and the features in the Gaussian convolved image represent the observation physical angular resolution.

The above-described process is what the CLEAN algorithm does. It creates a model from the observation, subtracts the visibility function of the model from the data to obtain the residuals, and then adds the residuals to the Gaussian convolved model. A diagram of this image reconstruction algorithm is shown in **Figure 2**:

<img src="./imaging_procedure.png">
<center> Figure 2: A model is convolved with a Gaussian to erase the spatial information of scales smaller than the angular resolution. Afterwards, it is added to the inverse Fourier Transform of the data minus the model, creating the final interferometric image. For a simulated model, there is no residual to be added, therefore it can immediately be convolved with a Gaussian and generate the synthetic observation.</center>

## The angular resolution is not uniquely determined

The angular resolution of an observation is not uniquely determined, as it is calculated from a weighted average of the baselines. Depending on how you weight the baselines of your observation (more weight to high S/N baselines, or more weight to extended baselines), the angular resolution of your reconstructed image can change dramatically.

A good example of the angular resolution change is the observation of the circumbinary disk CS Cha ([Kurtovic et al. 2022](https://ui.adsabs.harvard.edu/abs/2022arXiv220604427K/abstract)). This disk was observed by ALMA during 6.5hrs in total, in the timespan of a week. One way to change the weight of the baselines is to change a variable called **robust parameter**, which is a parameter of the CLEAN algorithm. Here in **Figure 3** you can see different images for the *same disk*, with the *same observation*, but with different robust parameters.

<img src="./cscha_continuum.png">
<center> Figure 3: Images of CS Cha continuum emission from the same ALMA observation, imaged with different robust parameters shown in the upper left corner of each panel. The white ellipse at the bottom left corner shows the angular resolution of each image, while the numbers next to it show the half-width of the representative Gaussian used to convolve the model, in units of milliarcsec. The scale bar represents 20au at the distance of the source.</center>

Let us say you have a radiative transfer model of this circumbinary disk, and you want to compare it with the observation. The angular resolution changes by a factor of $\approx 3$ between the images shown in the gallery, and all of them are valid images of the observation. If you were convolving your model with a Gaussian, **which angular resolution would you use to compare?**  

## Model reconstruction is a fundamental step of image reconstruction.

When comparing a simulation or a synthetic model to an interferometric observation, the Gaussian convolution is a good approximation of one of the steps of the imaging reconstruction algorithm, as shown in **Figure 2**. However, a fundamental step is the construction of the model itself, which will be dependent on the uv-coverage and the quality of the data.

**The most robust way to compare simulations to observations is by calculating the Fourier Transform of the simulated image at the same frecuencies measured by the uv-coverage of a given observation**. Afterwards, we can apply an image reconstruction algorithm and unveil what structures and properties would have been observed by a particular observational setup, and a particular image reconstruction. 